In [3]:
import os
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
from huggingface_hub import login
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import pipeline
import torch

In [4]:
torch.random.manual_seed(0)

In [5]:
login(token=os.environ['HF_TOKEN'])

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\User\.cache\huggingface\token
Login successful


In [ ]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", device_map='cuda', torch_dtype="auto" )
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [7]:
es_client = Elasticsearch('http://localhost:9200')
index_name = "homework-questions"

In [8]:
def es_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [9]:
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()
    
    prompt_template = """
QUESTION: {question}

CONTEXT:
{context}

ANSWER: 
""".strip()
    context = ""
    
    for doc in search_results:
        context = context + context_template.format(question = doc['question'], text = doc['text']) + "\n\n"
        
        
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [10]:
def llm(prompt):
    response = generator(prompt, max_length=500, num_return_sequences=1, truncation=True)
    response_final = response[0]['generated_text']
    return response_final[len(prompt):].strip()

In [14]:
def rag(query):
    search_results = es_search(query)
    prompt = build_prompt(query, search_results)

    return llm(prompt)

In [15]:
query = 'may I still enroll if the course has already started?'

In [16]:
rag(query)
#llm(query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


QUESTION: may I still enroll if the course has already started?

CONTEXT:
Q: The course has already started. Can I still join it?
A: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.

Q: When I plotted using Matplot lib to check if median has a tail, I got the error below how can one bypass?
A: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

Q: I don't know math. Can I take the course?
A: Yes! We'll cover some linear algebra in the course, but in general, there will be very few formulas, mostly code.
Here are some interesting videos covering linear algebra that you can already w

''